In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_save_path = "/content/drive/MyDrive/your_model_directory"

# Loading the model and tokenizer from the local path
model = AutoModelForSeq2SeqLM.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(model_save_path)


In [ ]:
!pip install py7zr

In [ ]:
from datasets import load_dataset

dataset_samsum = load_dataset("samsum", trust_remote_code=True)

test_dataset = dataset_samsum['test']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Checking if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

model_save_path = "/content/drive/MyDrive/your_model_directory"

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_save_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_save_path)


In [1]:
# Function to generate summaries
def generate_summary(batch):

    inputs = tokenizer(batch["dialogue"], truncation=True, padding="longest", return_tensors="pt").to(device)

    # Generate summaries
    summary_ids = model_pegasus.generate(
        inputs["input_ids"].to(device),
        max_length=128,
        min_length=32,
        num_beams=8,
        length_penalty=0.8
    )
  # Decode summaries
    batch["predicted_summary"] = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    return batch

# Generate summaries for the test dataset
predicted_test_dataset = test_dataset.map(generate_summary, batched=True, batch_size=8)


NameError: name 'test_dataset' is not defined

In [ ]:
# Saving the generated summaries and results for further analysis
predicted_test_dataset.to_csv("predicted_summaries.csv")


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

651993

In [ ]:
from google.colab import files

files.download("predicted_summaries.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

import pandas as pd

df = pd.DataFrame(predicted_test_dataset)

num_examples = 5
for i in range(num_examples):
    print(f"Example {i+1}:")
    print(f"Dialogue: {df.iloc[i]['dialogue']}")
    print(f"Reference Summary: {df.iloc[i]['summary']}")
    print(f"Generated Summary: {df.iloc[i]['predicted_summary']}")
    print("\n" + "-"*80 + "\n")


Example 1:
Dialogue: Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
Reference Summary: Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
Generated Summary: Amanda can't find Betty's number. Larry called Betty last time they were at the park together. Hannah wants Amanda to text Larry. Amanda will text Larry.

--------------------------------------------------------------------------------

Example 2:
Dialogue: Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks t

In [ ]:
!pip install rouge_score



In [ ]:
from datasets import load_metric
import numpy as np

# Loading the ROUGE metric
rouge_metric = load_metric("rouge")

def compute_metrics(predictions, references):
    results = rouge_metric.compute(predictions=predictions, references=references)

    formatted_results = {
        'rouge1': [results['rouge1'].mid.fmeasure] * len(predictions),
        'rouge2': [results['rouge2'].mid.fmeasure] * len(predictions),
        'rougeL': [results['rougeL'].mid.fmeasure] * len(predictions),
        'rougeLsum': [results['rougeLsum'].mid.fmeasure] * len(predictions)
    }
    return formatted_results

# Defining the function for `map`
def map_func(batch):

    predictions = batch["predicted_summary"]
    references = batch["summary"]

    metrics = compute_metrics(predictions, references)
    return metrics

metrics_result = predicted_test_dataset.map(map_func, batched=True)

# Print results
print(metrics_result)


Dataset({
    features: ['id', 'dialogue', 'summary', 'predicted_summary', 'rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
    num_rows: 819
})


In [ ]:
from datasets import load_metric

# Load the ROUGE metric
rouge_metric = load_metric("rouge")

# Defining a function to compute ROUGE scores
def compute_metrics(predictions):

    references = predictions["summary"]
    predictions = predictions["predicted_summary"]

    # Compute ROUGE scores
    results = rouge_metric.compute(predictions=predictions, references=references)
    return results

results = compute_metrics({
    "summary": predicted_test_dataset["summary"],
    "predicted_summary": predicted_test_dataset["predicted_summary"]
})

# Print the results
print("ROUGE Scores:")
for key, value in results.items():
    print(f"{key}: {value}")


ROUGE Scores:
rouge1: AggregateScore(low=Score(precision=0.371239256616925, recall=0.5733181868072524, fmeasure=0.42762341879405424), mid=Score(precision=0.3834689918925034, recall=0.5870362006552396, fmeasure=0.4369927111912), high=Score(precision=0.3950688000847977, recall=0.60011388953681, fmeasure=0.44638599579455657))
rouge2: AggregateScore(low=Score(precision=0.17117355617346705, recall=0.2723201090200402, fmeasure=0.19702222581084347), mid=Score(precision=0.18112728281736046, recall=0.28567507637060585, fmeasure=0.20733644880212732), high=Score(precision=0.19072015151046917, recall=0.3003447283398825, fmeasure=0.21748223087538263))
rougeL: AggregateScore(low=Score(precision=0.2931139917821006, recall=0.46061748210376113, fmeasure=0.3390546817133027), mid=Score(precision=0.3031733865593772, recall=0.47320530424151097, fmeasure=0.3479062081801377), high=Score(precision=0.3129978473043572, recall=0.48626540051061806, fmeasure=0.3564285904159457))
rougeLsum: AggregateScore(low=Score